# Mathematische Modelle zur Beschreibung von Daten

Anhand des Datensatzes (https://datahub.io/core/global-temp#data) wollen wir zwei grundsätzlich verschiedene Vorhersagen miteinander vergleichen:

K-Neighbors-Regression

Lineare-Regression

---

In [ ]:
import numpy as np
import pandas as pd
import scipy, scipy.stats
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
data = pd.read_json("../data/temperature/monthly_json.json") 
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
data.head()

Zunächst will ich die Daten etwas umformatieren, so dass das Datum zum index des Dataframes wird und die verschiedenen Quellen (Source) als Spalten dargestellt werden. Dazu nutze ich die Pivot-Funktion von pandas:

In [ ]:
pivot_df = data.pivot(index='Date', columns='Source', values='Mean')

In [ ]:
pivot_df.head()

Wie ich aus der Beschreibung der Daten weiß, handelt es sich um einen zeitlichen Verlauf der Temperaturmittelwerte. Diesen will ich zunächst einmal visualisieren um ein gefühl dafür zu bekommen.

In [ ]:
GCAG_data = pivot_df['GCAG'].plot(figsize=(20, 8))

In [ ]:
GCAG_data = pivot_df['GCAG']

X = GCAG_data.index.values.astype(np.int64) / 1000000000 
X = X.reshape(-1, 1)
X

In [ ]:
y = pivot_df['GCAG'].values

---

Aufgabe:
- Implementiere Logistische Regression
- Teile die Daten in ein Train- und Test-set
- Berechne Trainings- und Test- Fehler für KNR und Lineare Regression